<a href="https://colab.research.google.com/github/Andreusori/DataAnalytics-EDEM/blob/master/Ejercicios_Sesio%CC%81n_13_00_Introduccio%CC%81n_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerrequisites

Installing Spark and Apache Kafka Library in VM


---



In [1]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q https://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install py4j

# For maps
!pip install folium
!pip install plotly

     |████████████████████████████████| 204kB 8.2MB/s 


Define the environment (Java & Spark homes)

---

In [2]:
!ls /content

sample_data  spark-3.0.1-bin-hadoop3.2	spark-3.0.1-bin-hadoop3.2.tgz


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Starting Spark Session and print the version


---


In [4]:
import findspark
findspark.init("spark-3.0.1-bin-hadoop3.2")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.ui.port", "4500") \
        .getOrCreate()

spark.version

'3.0.1'

In [5]:
spark

In [6]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

Creating ngrok tunnel to allow Spark UI (Optional)


In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4500 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-11-28 08:20:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.225.137.4, 52.1.26.21, 54.221.249.251, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.225.137.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.3MB/s    in 0.7s    

2020-11-28 08:20:30 (18.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


# Descargar Datasets

In [8]:
!mkdir -p /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/frankenstein.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/el_quijote.txt -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/characters.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/planets.csv -P /dataset
!ls /dataset

characters.csv	el_quijote.txt	frankenstein.txt  planets.csv


# RDD

---



## Ejemplo 1

In [ ]:
!head /dataset/frankenstein.txt

FRANKENSTEIN

Or, The Modern Prometheus

by Mary Wollstonecraft (Godwin) Shelley

Letter 1

St. Petersburgh, Dec. 11th, 17—



esto lo ha hecho subre un fichero loacl

In [ ]:
textFile = spark.sparkContext.textFile("/dataset/frankenstein.txt")
textFile.first()
textFile.count()

7237

Abre la Spark UI e investiga qué ha sucedido


---



esto lo ha hecho sobre varios bloques de codigo y se ha ejectutado en paralelamente en todos los bloques. este ejemplo es muy chorra, xo en algo mas complejo, nos sirve xa dividir el proceso y que sea mucho más lijero y agil.


## Creación de colecciones paralelizadas
Una manera muy rápida de crear RDD desde la shell, cuando estamos aprendiendo, es crear una colección paralelizada. Para ello:

## Ejemplo 2

In [ ]:
distData = spark.sparkContext.parallelize([25, 20, 15, 10, 5])
distData.reduce(lambda x ,y: x + y)

75

¿De qué tipo es la variable distData?


## Ejercicio 1
Cuenta el número de líneas del fichero "el_quijote.txt"

---



In [ ]:
textFile2 = spark.sparkContext.textFile("/dataset/el_quijote.txt")
textFile2.count()

2186

In [ ]:
quijote_rdd = spark \
.sparkContext \
.textFile("/dataset/el_quijote.txt")

## Ejercicio 2
Imprime la primera línea del fichero "el_quijote.txt"

---



In [ ]:
textFile2.first()

'DON QUIJOTE DE LA MANCHA'

## Transformaciones y Acciones sobre RDDs 

### Acciones

### Ejemplo 3

In [ ]:
print(textFile.count()) # Número de elementos en el RDD
print(textFile.first()) # Primer elemento del RDD

2186
DON QUIJOTE DE LA MANCHA


### Transformaciones

### Ejemplo 4

In [ ]:
# ReduceByKey
lines = spark.sparkContext.textFile("/dataset/frankenstein.txt")
pairs = lines.map(lambda s: (s, 1)) #entoy creando un par clave/valor, a cada linea, le estoy asignando un 1 como valor
counts = pairs.reduceByKey(lambda a, b: a + b).cache() # 1º agrupa x la clave y después suma sus valores
counts.count()
counts.collect() #es como el show, xa k te muestre lo k ha ejecutado

In [ ]:
# SortByKey
sorted = counts.sortByKey() #sort = ordenar / en este caso como la Key es la linea, nos ordena todas las frases (lineas) por orden alfabetico
sorted.collect()

Abre la Spark UI e investiga que ha ido pasando.

### Ejemplo 5

In [ ]:
# Filter

linesWithSpark = textFile.filter(lambda line: "the" in line) #filtra todas las lineas en las q aparezca "the"
 linesWithSpark.count()

3712

### Ejercicio 3
Obtén el número de ocurrencias de cada palabra del fichero "frankenstein.txt"

In [12]:
textFile = spark.sparkContext.textFile("/dataset/frankenstein.txt")
contarPalabras = textFile.flatMap(lambda linea: linea.split(" ")).countByValue()
print(contarPalabras)

defaultdict(<class 'int'>, {'FRANKENSTEIN': 1, '': 3488, 'Or,': 1, 'The': 256, 'Modern': 1, 'Prometheus': 1, 'by': 441, 'Mary': 1, 'Wollstonecraft': 1, '(Godwin)': 1, 'Shelley': 1, 'Letter': 4, '1': 2, 'St.': 5, 'Petersburgh,': 2, 'Dec.': 1, '11th,': 1, '17—': 7, 'TO': 2, 'Mrs.': 4, 'Saville,': 4, 'England': 11, 'You': 52, 'will': 183, 'rejoice': 1, 'to': 2072, 'hear': 15, 'that': 974, 'no': 146, 'disaster': 2, 'has': 61, 'accompanied': 9, 'the': 3897, 'commencement': 5, 'of': 2634, 'an': 208, 'enterprise': 1, 'which': 540, 'you': 396, 'have': 356, 'regarded': 6, 'with': 650, 'such': 72, 'evil': 16, 'forebodings.': 1, 'I': 2720, 'arrived': 22, 'here': 23, 'yesterday,': 1, 'and': 2904, 'my': 1629, 'first': 102, 'task': 12, 'is': 296, 'assure': 4, 'dear': 62, 'sister': 7, 'welfare': 2, 'increasing': 1, 'confidence': 4, 'in': 1072, 'success': 6, 'undertaking.': 5, 'am': 114, 'already': 32, 'far': 38, 'north': 7, 'London,': 3, 'as': 477, 'walk': 4, 'streets': 4, 'feel': 40, 'a': 1338, 'col

Ahora hacemos lo mismo xo con el reduceByKey:
1º crearmos la var con todas las palabras
2º reduceByKey 

In [27]:
words = spark.sparkContext.textFile("/dataset/frankenstein.txt")

words.flatMap(lambda x: x.split(" ")) \
.map(lambda s: (s, 1)) \
.reduceByKey(lambda a, b: a + b) \




11606

**words.flatMap(lambda line: line.split(" ")) \**
Explicación funcion:
definimos los argumentos de la función: "lambda line:" (cada argumento de la función es una linea:) line.splie(" ") (separa cada linea en elementos distintos cada vez k se encuentra un espacio)

**.map(lambda word: (word, 1))** 
\ #asingnamos el valor 1 a cada palabra (clave) -> hemos creado un par (diccionario con solo 1 argumento: clave/valor)

.**reduceByKey(lambda a, b: a + b)**
aplicamos la funcion lamba y no un bucle fro xk "words" es un "rdd", x eso no 

### Ejercicio 4
Obtén el top-10 de palabras con más de 4 caracteres

---



In [34]:
words = spark.sparkContext.textFile("/dataset/frankenstein.txt")
words.flatMap(lambda s: s.split(" ")) \
.filter(lambda w: len(w) > 4) \
.map(lambda word: (word, 1)) \
.reduceByKey(lambda a,b: a+b) \
.map(lambda x: (x[1], x[0]) \
.sortByKey() \
.take(10) 

SyntaxError: ignored

**.map(lambda x: (x[1], x[0]) **
eso es xa inventir el Par (diccionario, donde esta la clave[0], -> pon el valor [1]

## Key/Value Pair RDD

---



### Ejemplo 6


---



In [37]:
charac_sw = spark.sparkContext.textFile("/dataset/characters.csv")
planets_sw = spark.sparkContext.textFile("/dataset/planets.csv")
charac_sw.take(10)

['name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,species',
 'Luke Skywalker,172,77,blond,fair,blue,19BBY,male,Tatooine,Human',
 'C-3PO,167,75,NA,gold,yellow,112BBY,NA,Tatooine,Droid',
 'R2-D2,96,32,NA,"white, blue",red,33BBY,NA,Naboo,Droid',
 'Darth Vader,202,136,none,white,yellow,41.9BBY,male,Tatooine,Human',
 'Leia Organa,150,49,brown,light,brown,19BBY,female,Alderaan,Human',
 'Owen Lars,178,120,"brown, grey",light,blue,52BBY,male,Tatooine,Human',
 'Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,Tatooine,Human',
 'R5-D4,97,32,NA,"white, red",red,NA,NA,Tatooine,Droid',
 'Biggs Darklighter,183,84,black,light,brown,24BBY,male,Tatooine,Human']

In [38]:
planets_sw.take(10)

['name;rotation_period;orbital_period;diameter;climate;gravity;terrain;surface_water;population',
 'Alderaan;24;364;12500;temperate;1 standard;grasslands, mountains;40;2000000000',
 'Yavin IV;24;4818;10200;temperate, tropical;1 standard;jungle, rainforests;8;1000',
 'Hoth;23;549;7200;frozen;1.1 standard;tundra, ice caves, mountain ranges;100;NA',
 'Dagobah;23;341;8900;murky;N/A;swamp, jungles;8;NA',
 'Bespin;12;5110;118000;temperate;1.5 (surface), 1 standard (Cloud City);gas giant;0;6000000',
 'Endor;18;402;4900;temperate;0.85 standard;forests, mountains, lakes;8;30000000',
 'Naboo;26;312;12120;temperate;1 standard;grassy hills, swamps, forests, mountains;12;4500000000',
 'Coruscant;24;368;12240;temperate;1 standard;cityscape, mountains;NA;1E+12',
 'Kamino;27;463;19720;temperate;1 standard;ocean;100;1000000000']

In [41]:
from itertools import islice

charac_sw_noheader = charac_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

planets_sw_noheader = planets_sw.mapPartitionsWithIndex(
    lambda idx, it: islice(it, 1, None) if idx == 0 else it)

codigo xa eliminar la cabezera del datasat, xk sino me lo coge como un una linea de valores

### Ejercicio 5
Obtén un listado con la población del planeta al que pertenece cada personaje de Star Wars


---


In [53]:
charac= charac_sw_noheader\
.map(lambda line: line.split(",")) \
.map(lambda w: (w[8], w[0])) \


In [54]:
planets= planets_sw_noheader\
.map(lambda line: line.split(";")) \
.map(lambda w: (w[0], w[8])) \


In [59]:
charac_planet = charac.join(planets)
print(charac_planet)

PythonRDD[116] at RDD at PythonRDD.scala:53


In [61]:
charac.join(planets) \
.map(lambda x: (x[1][0], x[0], x[1][1])) \
.collect()


[('Leia Organa', 'Alderaan', '2000000000'),
 ('Bail Prestor Organa', 'Alderaan', '2000000000'),
 ('Raymus Antilles', 'Alderaan', '2000000000'),
 ('Chewbacca', 'Kashyyyk', '45000000'),
 ('Tarfful', 'Kashyyyk', '45000000'),
 ('Lando Calrissian', 'Socorro', '300000000'),
 ('Wicket Systri Warrick', 'Endor', '30000000'),
 ('Darth Maul', 'Dathomir', '5200'),
 ('Ki-Adi-Mundi', 'Cerea', '450000000'),
 ('Saesee Tiin', 'Iktotch', 'NA'),
 ('Yarael Poof', 'Quermia', 'NA'),
 ('Jek Tono Porkins', 'Bestine IV', '62000000'),
 ('Palpatine', 'Naboo', '4500000000'),
 ('Jar Jar Binks', 'Naboo', '4500000000'),
 ('Roos Tarpals', 'Naboo', '4500000000'),
 ('Rugor Nass', 'Naboo', '4500000000'),
 ('Ric Olié', 'Naboo', '4500000000'),
 ('Quarsh Panaka', 'Naboo', '4500000000'),
 ('Gregar Typho', 'Naboo', '4500000000'),
 ('Cordé', 'Naboo', '4500000000'),
 ('Dormé', 'Naboo', '4500000000'),
 ('Padmé Amidala', 'Naboo', '4500000000'),
 ('Bossk', 'Trandosha', '42000000'),
 ('Mon Mothma', 'Chandrila', '1200000000'),
 ('F